<a href="https://colab.research.google.com/github/eppingera/EDGeS/blob/master/EDGe_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EDGe Pipeline
This pipeline trains a Machine Learning classifier to determine if an inputed district maps is gerrymandered. It is trained on 2016 House of Representatives election results and district demographic data. Here, we use 80% of the data as a testing set. After this model is trained, a drawn district map can be inputed (ie a list of districts with the demographic features). Then, the model will classify which districts are "gerrymandered."

#Import Data

In [0]:
#all the numbers written on census
import io
import pandas as pd #data frame
import numpy as np #handle numebrs

#to make this notebook's output stable across runs garantees this shuffle will be same as next
np.random.seed(42)
url = 'https://raw.githubusercontent.com/eppingera/EDGeS/master/data.csv'

#import data into pandas dataframe
data = pd.read_csv(url)

data source from:

https://hub.arcgis.com/datasets/ff48bbae433442a38f6c635b8c7baf72_0/data

https://www.politico.com/2016-election/results/map/house/

In [0]:
#View the first 5 entries
data.head()

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,POP75_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
0,1,101,Alabama - District 1,NaN,NaN,NaN,0,AL,717837,704571,583505,13266,118.3,280271,2.51,191687,3.04,345369,191383,88889,65098,0.80,0.89,0.74,44317,45363,46430,45111,46034,46924,45946,43501,44091,45066,49448,50869,46226,40900,30217,20893,...,40829,74045,47015,35059,32298,39552,51689,31280,27338,8699,7342,43658,60344,23880,747899,734633,606687,13266,123.3,293030,2.51,199572,3.04,363523,200427,92603,70493,0.82,0.89,0.81,1.89,0.93,1.97,16041,82074,153999,192299,125902,174921,70716
1,2,102,Alabama - District 2,134450.0,111640.0,NaN,246090,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,19410,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,NaN,284291,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,19115,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
3,4,104,Alabama - District 4,NaN,NaN,NaN,0,AL,694346,685737,574661,8609,78.1,275500,2.49,191464,3.00,318708,199856,75645,43208,0.27,0.25,0.10,41368,43227,43915,40784,39864,42698,42856,42385,44890,45242,48207,49374,46149,42070,31831,22207,...,45143,72622,50159,39932,34222,42111,48059,27250,22837,6042,4888,38746,53854,21593,701558,692949,579170,8609,78.9,278149,2.49,192436,3.01,323164,201763,76387,45015,0.21,0.19,0.10,1.67,0.19,1.17,10930,90091,145017,174948,126242,173988,74147
4,5,105,Alabama - District 5,204791.0,101577.0,NaN,306368,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,20376,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177


In [0]:
#get data type and number of instances for all the atributes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 94 columns):
NUMBER             436 non-null int64
ID                 436 non-null int64
NAME               436 non-null object
NUM_REP_VOTES      386 non-null float64
NUM_DEM_VOTES      394 non-null float64
NUM_OTH_VOTES      186 non-null float64
TOTAL_NUM_VOTES    436 non-null int64
ST_ABBREV          436 non-null object
TOTPOP_CY          436 non-null int64
HHPOP_CY           436 non-null int64
FAMPOP_CY          436 non-null int64
GQPOP_CY           436 non-null int64
POPDENS_CY         436 non-null float64
TOTHH_CY           436 non-null int64
AVGHHSZ_CY         436 non-null float64
FAMHH_CY           436 non-null int64
AVGFMSZ_CY         436 non-null float64
TOTHU_CY           436 non-null int64
OWNER_CY           436 non-null int64
RENTER_CY          436 non-null int64
VACANT_CY          436 non-null int64
POPGRW10CY         436 non-null float64
HHGRW10CY          436 non-null float64
FAM

**Note:**
1.   the NUM_REP_VOTES, NUM_DEM_VOTES, & NUM_OTH_VOTES categories are not full. That's not a problem because we are going to use these categories to create new columns that we will actually use. 
2.   The NAME and ST_ABBREV categories are objects. This is not a problem because we are going to drop them in the ML model anyway. 
3.   Otherwise, all categoires have 436 data numberical points.
4.   Also, there are only 435 voting members of the House of Representativies. This data set contains the non-voting DC representative.
5.   Complete list of attributes is in the first data source link above -- other than the NUM_REP_VOTES, NUM_DEM_VOTES, NUM_OTH_VOTES, TOT_NUM_VOTES (number of Republican, Democrate, other, and total votes for each district)

To summarize the features of the dataset, they are various metrics of: 


*   Age demographics and distribution
*   Socioeconomic statuses and distribution
*   Percent homeownership, renting, vacancy
*   Population density
*   Population growth rate



#Remove Uncontested Seats

Remove any row where the House of Representatives' seat was uncontested that year (aka we have not voting results, so we can't use the data point).

In [0]:
#remove_uncon() removes any row where the seat was not contested in 2016
#returns a new pandas data fram without uncontested districts

def remove_uncon():
  uncon = []
  for i in range(len(data)):
    if (data.iloc[i,6] == 0):
      uncon.append(i)
  return data.drop(uncon)

#data_2 now holds the new data frame without uncontested districts
data_2 = remove_uncon()

#just checking :)
data_2.head(15)
      

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,POP75_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,134450.0,111640.0,NaN,246090,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,19410,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,NaN,284291,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,19115,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,204791.0,101577.0,NaN,306368,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,20376,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,240897.0,81296.0,NaN,322193,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,17844,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,125729.0,90784.0,32262.0,248775,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,11425,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932
8,9,401,Arizona - District 1,100236.0,117048.0,13122.0,230406,AZ,756062,728175,603424,27887,13.7,263426,2.76,183822,3.28,346518,180385,83041,83092,1.01,0.99,0.86,53470,52282,52053,53245,54880,49064,45922,43514,42227,42194,46555,49804,48559,45283,32804,21196,...,33029,67054,43043,28679,27505,37155,49430,33667,28763,8204,6969,47550,62411,22226,803522,775636,640942,27886,14.6,280488,2.77,194863,3.29,369964,192427,88061,89476,1.23,1.26,1.17,2.00,1.30,2.14,15173,71722,177391,209832,119793,179623,69419
9,10,402,Arizona - District 2,157677.0,120572.0,NaN,278249,AZ,732755,715941,541731,16814,93.5,307006,2.33,183191,2.96,353133,183176,123830,46127,0.50,0.44,0.22,41676,41091,40954,41539,49767,51378,47434,42406,

#Remove NaN Values

One challenge with this data set is the NaN is the NUM Votes columns. We will use a simple method to remove these values and make them 0 instead.

In [0]:
data_2.fillna(0,inplace=True) #fill NaN values with 0
data_2.head(15)

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,POP75_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,134450.0,111640.0,0.0,246090,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,19410,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,0.0,284291,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,19115,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,204791.0,101577.0,0.0,306368,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,20376,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,240897.0,81296.0,0.0,322193,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,17844,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,125729.0,90784.0,32262.0,248775,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,11425,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932
8,9,401,Arizona - District 1,100236.0,117048.0,13122.0,230406,AZ,756062,728175,603424,27887,13.7,263426,2.76,183822,3.28,346518,180385,83041,83092,1.01,0.99,0.86,53470,52282,52053,53245,54880,49064,45922,43514,42227,42194,46555,49804,48559,45283,32804,21196,...,33029,67054,43043,28679,27505,37155,49430,33667,28763,8204,6969,47550,62411,22226,803522,775636,640942,27886,14.6,280488,2.77,194863,3.29,369964,192427,88061,89476,1.23,1.26,1.17,2.00,1.30,2.14,15173,71722,177391,209832,119793,179623,69419
9,10,402,Arizona - District 2,157677.0,120572.0,0.0,278249,AZ,732755,715941,541731,16814,93.5,307006,2.33,183191,2.96,353133,183176,123830,46127,0.50,0.44,0.22,41676,41091,40954,41539,49767,51378,47434,42406,

#Generate Labels

In [0]:
#rep_win() returns an int array 
#(0 if dem's win, 1 if rep's win, others won no seats)
#this will be added as a column to the pd data frame
def rep_win():
  returner = []
  for i in range(len(data_2)):
    if (data_2.iloc[i,3] > data_2.iloc[i,4]):
      returner.append(1)
    else:
      returner.append(0)
  return returner

In [0]:
#add REP_WIN as a Column to the pd datafram after TOTAL_NUM_VOTES
data_2.insert(7, "REP_WIN", rep_win(), True) 

In [0]:
#check to see if that worked....
data_2.head(20)

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,REP_WIN,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,134450.0,111640.0,0.0,246090,1,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,0.0,284291,1,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,204791.0,101577.0,0.0,306368,1,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,240897.0,81296.0,0.0,322193,1,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,125729.0,90784.0,32262.0,248775,1,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932
8,9,401,Arizona - District 1,100236.0,117048.0,13122.0,230406,0,AZ,756062,728175,603424,27887,13.7,263426,2.76,183822,3.28,346518,180385,83041,83092,1.01,0.99,0.86,53470,52282,52053,53245,54880,49064,45922,43514,42227,42194,46555,49804,48559,45283,32804,...,33029,67054,43043,28679,27505,37155,49430,33667,28763,8204,6969,47550,62411,22226,803522,775636,640942,27886,14.6,280488,2.77,194863,3.29,369964,192427,88061,89476,1.23,1.26,1.17,2.00,1.30,2.14,15173,71722,177391,209832,119793,179623,69419
9,10,402,Arizona - District 2,157677.0,120572.0,0.0,278249,1,AZ,732755,715941,541731,16814,93.5,307006,2.33,183191,2.96,353133,183176,123830,46127,0.50,0.44,0.22,41676,41091,40954,41539,49767,51378,47434,42406,40857,41343,47290,51143

In [0]:
#by_state() returns 3 lists
# 1) state abbreviations
# 2) total votes in each state
# 3) republican votes in each state
# 4) total seats in each state
# 5) republican seats in each state
def by_state():
  #list that will ultimately be returned
  state_abbrev = []
  total_votes = []
  rep_votes = []
  total_seats = []
  rep_seats = []

  #the 0th elements of the state array hols the first state abbrevatiation (AL)
  state_abbrev.append(data_2.iloc[0,8])
  total_votes.append(data_2.iloc[0,6])
  rep_votes.append(data_2.iloc[0,3])
  total_seats.append(1)
  rep_seats.append(data_2.iloc[0,7])

  #counter will be used to iterate through the returned arrays
  counter = 0

  for i in range(len(data_2)):
    #if the distric we are at is in the same state
    if (data_2.iloc[i,8] == state_abbrev[counter]):
      #add its values to the counter's position in the arrays below
      total_votes[counter] = total_votes[counter] + data_2.iloc[i,6]
      rep_votes[counter] = rep_votes[counter] + data_2.iloc[i,3]
      total_seats[counter] = total_seats[counter] + 1
      rep_seats[counter] = rep_seats[counter] + data_2.iloc[i,7]

    #if its a new state
    else:
      #add one to the counter
      counter = counter + 1
      #append its values to the ends of the returner arrays
      state_abbrev.append(data_2.iloc[i,8])
      total_votes.append(data_2.iloc[i,6])
      rep_votes.append(data_2.iloc[i,3])
      total_seats.append(1)
      rep_seats.append(data_2.iloc[i,7])
      #if there is another district in this state, this values will be increase accordingly
  
  return {
      "state_abbrev" : state_abbrev,
      "total_votes" : total_votes, 
      "rep_votes" : rep_votes,
      "total_seats" : total_seats, 
      "rep_seats" : rep_seats
  }    

In [0]:
#create a dictionary to hold the state_abbrev, total_votes, rep_votes, total_seats, rep_seats arrays
state_info = by_state()

In [0]:
data_2.head()

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,REP_WIN,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,134450.0,111640.0,0.0,246090,1,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,0.0,284291,1,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,204791.0,101577.0,0.0,306368,1,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,240897.0,81296.0,0.0,322193,1,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,125729.0,90784.0,32262.0,248775,1,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932


#HUGE: Setting the gerrymandering threshold
Here we can set the threshold for gerrymandering. Our metric for gerrymanding is the difference percent of seats won by a party and the percent of the total or overall vote that party won. The threshold is the difference in these percentages that is considered gerrymandering.

In [0]:
#let's set the threshold to 15% for now
threshold = 0.15

data_2.head()

,NUMBER,ID,NAME,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,REP_WIN,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,134450.0,111640.0,0.0,246090,1,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,190724.0,93567.0,0.0,284291,1,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,204791.0,101577.0,0.0,306368,1,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,240897.0,81296.0,0.0,322193,1,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,125729.0,90784.0,32262.0,248775,1,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932


In [0]:
#is_gerrymandered() returns whether or not a distric is gerrymandered
#it's based on the threshold set above
#this list will serve as the labeles for the ML model

def is_gerrymandered():
  is_gerrymandered = []
  for i in range(len(data_2)):
    #loc holds the index in the dictionary's arrays that we will access
    loc = (state_info['state_abbrev']).index(data_2.iloc[i,8])

    #percent_seats holds the #rep_seats / #total seats
    percent_seats = (state_info['rep_seats'])[loc] / (state_info['total_seats'])[loc]

    #percent_votes holds the #rep_votes / #total votes
    percent_votes = (state_info['rep_votes'])[loc] / (state_info['total_votes'])[loc]

    #if the difference in the %seats and %votes is greater than threshold, label 1
    if (abs(percent_seats - percent_votes) > threshold):
      is_gerrymandered.append(1)

    else:
      is_gerrymandered.append(0)

  return is_gerrymandered

In [0]:
labels = is_gerrymandered()
data_2.insert(3, "Labels", labels, True) 

#Quick aside

In [0]:
#For fun, let's see which states are the most gerrymandered :)
#negative: more gerrymandered for democrats
#postive: more gerrymandered for republicans

class State:
  abbrev = ''
  diff = 0

  #constructor
  def __init__(self, abbrev, diff):
    self.abbrev = abbrev
    self.diff = diff

  #toString method
  def __str__(self):
    return self.abbrev + ", " + str(self.diff)

def most():
  states = []
  #for each state, determine the difference in percent of votes and seats won
  for i in range(len((state_info['state_abbrev']))):
    percent_seats = (state_info['rep_seats'])[i] / (state_info['total_seats'])[i]
    percent_votes = (state_info['rep_votes'])[i] / (state_info['total_votes'])[i]

    abbrev = (state_info['state_abbrev'])[i]
    diff = percent_seats - percent_votes

    state = State(abbrev, diff)
    states.append(state)

  #for every object, compute lambda function (diff field)
  #sorts based on the diff attribute
  states.sort(key=lambda x: (x.diff))

  return states

for s in most():
  print(s)


NH, -0.44097509099435056
DE, -0.4096359188098708
CT, -0.36269804470608596
MA, -0.2518551199512158
MD, -0.2234346665446515
NV, -0.21221281909763456
HI, -0.20716408097856373
OR, -0.1878798960649744
NM, -0.10656724150083918
MN, -0.09247514969032128
CA, -0.08999454549518876
IL, -0.08043361392800258
NJ, -0.04821919298546046
WA, -0.046305255115762056
NY, -0.023720440695768397
VT, 0.0
ME, 0.018223473310051197
AZ, 0.06512458136515864
FL, 0.07931391699287915
CO, 0.08339266730186834
TX, 0.12520390143432447
LA, 0.12805988304038363
GA, 0.14513230266705845
KY, 0.15008436891663346
TN, 0.15288910574777292
MO, 0.1592253831531799
MI, 0.16207557664819877
VA, 0.17035186525661616
MS, 0.17102123315172646
OH, 0.17291763947421734
WI, 0.2084185737769978
IA, 0.21231180435742614
PA, 0.2170467858691194
NC, 0.2364900024996225
IN, 0.25617534131621733
SC, 0.2722437745558771
AR, 0.2878391182806851
ND, 0.30764207370467667
OK, 0.3255734769465888
ID, 0.34434523458942434
WV, 0.3514858019458711
AL, 0.3556644973210574
SD,

#Machine Learning Models

In [0]:
data_2.head()

,NUMBER,ID,NAME,Labels,NUM_REP_VOTES,NUM_DEM_VOTES,NUM_OTH_VOTES,TOTAL_NUM_VOTES,REP_WIN,ST_ABBREV,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,2,102,Alabama - District 2,1,134450.0,111640.0,0.0,246090,1,AL,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,3,103,Alabama - District 3,1,190724.0,93567.0,0.0,284291,1,AL,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,5,105,Alabama - District 5,1,204791.0,101577.0,0.0,306368,1,AL,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,6,106,Alabama - District 6,1,240897.0,81296.0,0.0,322193,1,AL,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,8,200,Alaska Congressional District,1,125729.0,90784.0,32262.0,248775,1,AK,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932


In [0]:
#Drop what we won't use in our ML model
data_2 = data_2.drop(["NUMBER", "ID", "NAME", "NUM_REP_VOTES", "NUM_DEM_VOTES", "NUM_OTH_VOTES", "ST_ABBREV"], axis=1) 

In [0]:
data_2.head()

,Labels,TOTAL_NUM_VOTES,REP_WIN,TOTPOP_CY,HHPOP_CY,FAMPOP_CY,GQPOP_CY,POPDENS_CY,TOTHH_CY,AVGHHSZ_CY,FAMHH_CY,AVGFMSZ_CY,TOTHU_CY,OWNER_CY,RENTER_CY,VACANT_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,POP0_CY,POP5_CY,POP10_CY,POP15_CY,POP20_CY,POP25_CY,POP30_CY,POP35_CY,POP40_CY,POP45_CY,POP50_CY,POP55_CY,POP60_CY,POP65_CY,POP70_CY,POP75_CY,POP80_CY,POP85_CY,MALES_CY,FEMALES_CY,NOHS_CY,...,WIDOWED_CY,DIVORCD_CY,HINC0_CY,HINC15_CY,HINC25_CY,HINC35_CY,HINC50_CY,HINC75_CY,HINC100_CY,HINC150_CY,HINC200_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,TOTPOP_FY,HHPOP_FY,FAMPOP_FY,GQPOP_FY,POPDENS_FY,TOTHH_FY,AVGHHSZ_FY,FAMHH_FY,AVGFMSZ_FY,TOTHU_FY,OWNER_FY,RENTER_FY,VACANT_FY,POPGRWCYFY,HHGRWCYFY,FAMGRWCYFY,PCIGRWCYFY,OWNGRWCYFY,MHIGRWCYFY,High_Inc,Low_Inc,GenZ_1,GenMil,GenX_1,GenBoom,GenSilent
1,1,246090,1,700843,678088,555536,22755,69.1,275117,2.46,184343,3.01,318462,182005,93112,43345,0.42,0.40,0.25,43286,43803,44050,43250,48376,49472,47585,43810,44250,44188,47352,47558,43355,37929,27845,19410,12958,12366,342724,358119,25268,...,40989,73934,48874,34568,32701,39918,48450,29964,27023,8226,5387,41838,57535,23075,709892,687137,561345,22755,70.0,278598,2.47,185833,3.02,324401,184375,94223,45803,0.26,0.25,0.16,1.69,0.26,1.68,13613,83442,147721,198552,124496,163829,66245
2,1,284291,1,717877,698711,565972,19166,95.2,282710,2.47,187981,3.01,333923,192327,90384,51213,0.80,0.76,0.59,42378,43667,43919,48366,59901,47279,45566,44032,44980,45245,48032,49112,44941,39691,28317,19115,12365,10975,352158,365723,26639,...,40240,72482,53231,37514,34118,39908,49024,29889,26018,7638,5361,40059,55791,22365,744012,724846,584538,19166,98.6,292725,2.48,193421,3.02,347074,198480,94245,54349,0.72,0.70,0.57,1.62,0.63,1.25,12999,90745,147038,210477,126777,169330,64252
4,1,306368,1,721268,705942,582285,15326,196.1,289053,2.44,194767,2.99,320106,199815,89238,31053,0.88,0.90,0.74,41275,43801,45749,45964,47550,47221,47358,44956,46091,48209,53591,52076,44622,38130,28839,20376,13512,11947,354789,366478,21915,...,37595,73410,39616,33068,30905,38985,49777,33099,38766,14554,10282,50659,69509,28197,750701,735375,604824,15326,204.1,301036,2.44,201874,3.00,334466,208302,92733,33430,0.80,0.82,0.72,1.82,0.84,1.81,24836,72684,148287,197622,133395,173788,68177
5,1,322193,1,709464,697834,586005,11630,170.1,275397,2.53,191244,3.06,305432,204846,70551,30035,0.61,0.55,0.41,42919,45845,47534,43781,42308,45801,48879,48751,49135,47314,49161,48524,43388,36872,26026,17844,12554,12827,346526,362937,17351,...,35844,66592,26850,25262,26456,36482,50695,36269,42324,16603,14456,58820,81627,31943,732928,721299,604255,11629,175.7,283947,2.54,196372,3.08,315318,211279,72668,31371,0.65,0.61,0.53,1.71,0.62,2.12,31059,52112,154136,192150,135548,164428,63197
7,1,248775,1,752680,724373,575297,28307,1.3,270970,2.67,177298,3.24,323266,166065,104905,52296,0.93,0.78,0.60,54324,52305,51093,48683,57712,59917,56010,48792,45941,46853,53109,54738,45850,33433,19950,11425,6796,5749,391026,361654,12107,...,21396,70547,18849,18157,19113,31407,51345,39153,53777,22145,17023,72692,91524,34264,779291,750984,593821,28307,1.4,279674,2.69,181970,3.26,334901,171886,107788,55227,0.70,0.63,0.52,1.44,0.69,2.07,39168,37006,177085,232791,132460,171412,38932


In [0]:
#Create a test set of data to work with.
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data_2, test_size=0.2, random_state=42)

#create labels and features for both the training and testing sets
x_train = train_set.drop("Labels", axis = 1)
y_train = train_set["Labels"]

x_test = test_set.drop("Labels", axis = 1)
y_test = test_set["Labels"]

##SGD Classifier

In [0]:
#SGDClassifier (Stochastic Gradient Descent):

#Import the SGDClassifier
from sklearn.linear_model import SGDClassifier

#Import the cross_val_score
from sklearn.model_selection import cross_val_predict

#Define the classifier with max_iter = 5, tol = -np.infty, random_state = 42
sgd_clf = SGDClassifier(max_iter = 5, tol = -np.infty, random_state = 42)

#Fit the sgd classifer
sgd_clf.fit(x_train, y_train)

#hold the sgd predictions
sgd_train_pred = cross_val_predict(sgd_clf, x_train, y_train, cv = 10)

In [0]:
#SGD F1 Score
from sklearn.metrics import f1_score

f1_score(y_train, sgd_train_pred)

0.5529411764705883

In [0]:
#SGD ROC AUC
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, sgd_train_pred)

0.5406060606060606

##SVC Classifier

In [0]:
#SVCClassifier (Support Vector Classification)
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_clf.fit(x_train, y_train)
svm_train_pred = cross_val_predict(svm_clf, x_train, y_train, cv=10)

In [0]:
#SVC F1 Score
f1_score(y_train, svm_train_pred)

0.0

In [0]:
#SVC ROC AUC Score
roc_auc_score(y_train, svm_train_pred)

0.5

##Random Forest Classifier

In [0]:
#Random Forest:

#Import the RandomForestClassifier:
from sklearn.ensemble import RandomForestClassifier

#Define the classifier with n_estimators = 100 and random_state = 42 and
forest_clf = RandomForestClassifier(n_estimators = 100, random_state = 42)

#Fit the classifier to X_train and y_train
forest_clf.fit(x_train, y_train)

#Get scores for the random forest classifier, cv = 10 and scoring 
forest_train_pred = cross_val_predict(forest_clf, x_train, y_train, cv=10)

In [0]:
#Forest F1 Score
f1_score(y_train, forest_train_pred)

0.7919463087248321

In [0]:
#Fores ROC AUC
roc_auc_score(y_train, forest_train_pred)

0.8081060606060606

#Test Random Forest Model
The Randcom Forest Classifier appears to be best for this dataset. Now, we can test the classifier on the testing data and measure its accuracy.

In [0]:
#test Random Forest classifier on the test data set
y_test_pred = forest_clf.predict(x_test)

In [0]:
#F1 score for test data set
f1_score(y_test, y_test_pred)

0.8648648648648649

In [0]:
#ROC AUC Score for test data
roc_auc_score(y_test, y_test_pred)

0.8768768768768769

#End Result:

According to this finding.... https://www.sciencedirect.com/science/article/pii/S1556086415306043

The Random Forest ROC AUC Value is 0.88, which considered "excellent" for an ML model.